In [8]:
import json
import os

# Ruta al archivo de configuración JSON
config_file_path = '../cfg-data/env-config.json'

def read_config(filename):
    with open(filename, 'r') as f:
        data = json.load(f)
        return data

# Leer la configuración
config_data = read_config(config_file_path)

# Extraer los parámetros necesarios
INFLUXDB_IP = config_data['env']['INFLUXDB_IP']
INFLUXDB_DATABASE = config_data['env']['INFLUXDB_DATABASE']


In [1]:
%pip install pandas
%pip install influxdb

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [9]:
import pandas as pd
from influxdb_client import InfluxDBClient

# Configuración de InfluxDB
# Deben ser identicos a los del *.yml
INFLUXDB_URL = 'http://localhost:8086'  
INFLUXDB_TOKEN = 'my-secret-token'       
INFLUXDB_ORG = 'SEAT S.A.'               
INFLUXDB_BUCKET = 'my-bucket'            

# Consulta Flux
query = '''
from(bucket: "my-bucket")
  |> range(start: -1h)  // Consulta datos de la última hora
  |> filter(fn: (r) => r._field =~ /.*POR_OEE|.*POR_Disponibilidad|.*POR_Calidad
|.*POR_Rendimiento|.*T_Paro_No_Programado|.*T_Neto|.*T_Ciclo_ACT|.*T_Ciclo_Medio|.*T_LOG_IN_ACT|.*T_LOG_OUT_ACT|.*T_Ciclo_Logistico|.*T_Averia_ACT|.*N_SOLL|.*N_TREND|.*N_PROD|.*N_OK|.*N_NOK|.*N_PAUSA|.*ST_Instal_en_seq|.*ST_Instal_en_PAUSA|.*ST_Instal_en_BREAK|.*ST_EnCiclo_CO|.*ST_Averia|.*ST_IN_LOG_IN|.*ST_IN_LOG_OUT|.*SP_TC_OBJ|.*SP_SOLL/)
  |> limit(n: 1000)  // Limitar a los primeros 1000 resultados
'''

# Crear el cliente de InfluxDB
client = InfluxDBClient(url=INFLUXDB_URL, token=INFLUXDB_TOKEN, org=INFLUXDB_ORG)

# Ejecutar la consulta
query_api = client.query_api()
result = query_api.query(org=INFLUXDB_ORG, query=query)

# Convertir el resultado en un DataFrame de pandas
records = []
for table in result:
    for record in table.records:
        # Extraer solo las columnas que necesitas
        records.append({
            '_time': record.get_time(),
            '_field': record.get_field(),
            '_value': record.get_value()
        })

# Crear DataFrame con las columnas deseadas
df = pd.DataFrame(records)

# Convertir la columna '_time' a tipo datetime
df['_time'] = pd.to_datetime(df['_time'])

# Buscar filas duplicadas basadas en '_time' y '_field'
# y conservar solo la última entrada para cada combinación
df = df.drop_duplicates(subset=['_time', '_field'], keep='last')

# Pivotar el DataFrame para tener una columna por cada '_field'
df_pivot = df.pivot(index='_time', columns='_field', values='_value')

# Rellenar los valores NaN con el último valor válido
df_filled = df_pivot.fillna(method='ffill')

# Agrupar por el índice '_time' (sin truncar los segundos) y consolidar en una única fila por timestamp
df_grouped = df_filled.groupby(df_filled.index).first().reset_index()

# Guardar el DataFrame agrupado a un archivo CSV
df_grouped.to_csv('data/data_grouped.csv', index=False)

# Cerrar el cliente de InfluxDB
client.close()


_field                            _time  \
0      2024-09-15 15:32:32.410211+00:00   
1      2024-09-15 15:32:32.410221+00:00   
2      2024-09-15 15:32:47.412738+00:00   
3      2024-09-15 15:32:47.412740+00:00   
4      2024-09-15 15:33:02.424098+00:00   
..                                  ...   
827    2024-09-15 16:19:12.797896+00:00   
828    2024-09-15 16:19:12.797897+00:00   
829    2024-09-15 16:19:12.797909+00:00   
830    2024-09-15 16:19:12.797910+00:00   
831    2024-09-15 16:19:12.797913+00:00   

_field  120210R01_FB_SEAT_Statistik_ZAU.120210R01.N_NOK  \
0                                                   NaN   
1                                                   NaN   
2                                                   NaN   
3                                                   NaN   
4                                                   NaN   
..                                                  ...   
827                                                 0.0   
828        

/tmp/ipykernel_5538/3580975593.py:51: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_filled = df_pivot.fillna(method='ffill')
